### Create Artist's Birth Place Map over time ###

In [20]:
# import of the libraries
# !pip install mapclassify
# !pip install folium

In [21]:
import pandas as pd
import re

artist_csv = "../output_csv/bremen_data_v5_merge_artist.csv"

df_artist = pd.read_csv(artist_csv)
df_artist.head(10)

,artistGNDId,artistName (preferred),placeOfBirth,birth year,num_works,birth_year_span_start
0,100076262,"Caylus, Anne Claude Philippe de",Paris,1692.0,24,1650.0
1,100104401,"Dequevauviller, François",Abbeville,1745.0,1,1700.0
2,100648916,"Collas, Achille",Paris,1795.0,3,1750.0
3,100786294,"Porret, Henri Désiré",Lille,1800.0,6,1800.0
4,100811833,"Kraus, Johann Ulrich",Augsburg,1655.0,48,1650.0
5,100811957,Anonym,Haarlem,1537.0,1,1500.0
6,100811957,"Anonym, niederländisch, 16. Jahrhundert",Haarlem,1501.0,17,1500.0
7,100811957,"Cleve III., Hendrik van",Haarlem,1525.0,2,1500.0
8,100811957,"Galle, Philippe",Haarlem,1537.0,56,1500.0
9,1011377616,"Loutrel, Victor Jean Baptiste",Antwerpen,1821.0,1,1800.0


In [22]:
total_num = df_artist.shape[0]
count_anonym = df_artist[df_artist['artistName (preferred)'].str.contains("Anonym", na=False)].shape[0]

print(f"Total number of artists in Bremen dataset: {total_num}")
print(f"Number of anonymous artists: {count_anonym} which is {count_anonym/total_num*100}%")

Total number of artists in Bremen dataset: 1402
Number of anonymous artists: 230 which is 16.405135520684734%


In [25]:
(df_artist["placeOfBirth"].isna() | (df_artist["placeOfBirth"] == "")).sum()

np.int64(0)

### Convert birth place to longitude and latitude ###

In [26]:
# pip install pandas geopy
# !pip install shapely

In [ ]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from shapely.geometry import Point
import pandas as pd
import numpy as np

# Set up geocoder
geolocator = Nominatim(user_agent="geoapi")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Prepare latitude and longitude columns


len(df_artist["placeOfBirth"].unique())

402

In [31]:
city_locations_dict = {}

# Loop through each place and get coordinates
for place in df_artist["placeOfBirth"].unique():
    try:
        location = geocode(place)

        if location:
            city_locations_dict[place] = {"latitude": location.latitude, "longitude": location.longitude}
    except Exception as e:
        print(f"Error with place '{place}': {e}")

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Málaga',), **{}).
Traceback (most recent call last):
  File "c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connection.py", line 507, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\http\client.py", line 1428, in getresponse
    response.begin()
  File "c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\http\client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\http\client.py", line 292, in _read_status
    line = str(s

In [33]:
print(city_locations_dict)

{'Paris': {'latitude': 48.8588897, 'longitude': 2.320041}, 'Abbeville': {'latitude': 50.1060835, 'longitude': 1.8337029}, 'Lille': {'latitude': 50.6365654, 'longitude': 3.0635282}, 'Augsburg': {'latitude': 48.3690341, 'longitude': 10.8979522}, 'Haarlem': {'latitude': 52.3837058, 'longitude': 4.6435597}, 'Antwerpen': {'latitude': 51.2211097, 'longitude': 4.3997081}, 'Gorinchem': {'latitude': 51.8296133, 'longitude': 4.9738743}, 'Nantes': {'latitude': 47.2186371, 'longitude': -1.5541362}, 'Tours': {'latitude': 47.3900474, 'longitude': 0.6889268}, 'Prag': {'latitude': 50.0596288, 'longitude': 14.4464593}, 'Wien': {'latitude': 48.2083537, 'longitude': 16.3725042}, 'Caen': {'latitude': 49.1813403, 'longitude': -0.3635615}, 'Rouen': {'latitude': 49.4404591, 'longitude': 1.0939658}, 'Rotterdam': {'latitude': 51.9244424, 'longitude': 4.47775}, 'Maidstone': {'latitude': 51.2748258, 'longitude': 0.5231646}, 'Lodève': {'latitude': 43.73106, 'longitude': 3.31892}, 'Schaffhausen': {'latitude': 47.

In [34]:
df_artist["latitude"] = df_artist["placeOfBirth"].apply(lambda x: city_locations_dict.get(x, {}).get("latitude"))
df_artist["longitude"] = df_artist["placeOfBirth"].apply(lambda x: city_locations_dict.get(x, {}).get("longitude"))

df_artist.head(10)

,artistGNDId,artistName (preferred),placeOfBirth,birth year,num_works,birth_year_span_start,latitude,longitude
0,100076262,"Caylus, Anne Claude Philippe de",Paris,1692.0,24,1650.0,48.858890,2.320041
1,100104401,"Dequevauviller, François",Abbeville,1745.0,1,1700.0,50.106083,1.833703
2,100648916,"Collas, Achille",Paris,1795.0,3,1750.0,48.858890,2.320041
3,100786294,"Porret, Henri Désiré",Lille,1800.0,6,1800.0,50.636565,3.063528
4,100811833,"Kraus, Johann Ulrich",Augsburg,1655.0,48,1650.0,48.369034,10.897952
5,100811957,Anonym,Haarlem,1537.0,1,1500.0,52.383706,4.643560
6,100811957,"Anonym, niederländisch, 16. Jahrhundert",Haarlem,1501.0,17,1500.0,52.383706,4.643560
7,100811957,"Cleve III., Hendrik van",Haarlem,1525.0,2,1500.0,52.383706,4.643560
8,100811957,"Galle, Philippe",Haarlem,1537.0,56,1500.0,52.383706,4.643560
9,1011377616,"Loutrel, Victor Jean Baptiste",Antwerpen,1821.0,1,1800.0,51.221110,4.399708


In [35]:
output_csv = "../output_csv/bremen_data_v5_2_with_coordinate.csv"

df_artist.to_csv(output_csv, index=False, encoding="utf-8")

In [3]:
import folium
import pandas as pd

coordinative_csv = "../output_csv/bremen_data_v5_2_with_coordinate.csv"
df_artist_with_coordinate = pd.read_csv(coordinative_csv) 

# Create a base map
map_ = folium.Map(location=[52, 10], zoom_start=5)

# Add artist birthplaces
for _, row in df_artist_with_coordinate.dropna(subset=["latitude", "longitude"]).iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=row.get("artistName (preferred)", "Unknown")
    ).add_to(map_)

map_

